In [ ]:
from tqdm import tqdm
from utils.dataset import AI4VN_AirDataLoader
from utils.evaluation import eval_regression_model

air_data_loader = AI4VN_AirDataLoader()

## Scikit-learn ensemble models

In [ ]:
from utils.models import EnsembleModel

### Train models

In [ ]:
X_train, X_test, y_train, y_test = air_data_loader.get_data_loader_sklearn()

In [ ]:
model_gb = EnsembleModel(name="gradientboosting")
model_rf = EnsembleModel(name="randomforest")

trained_model_gb = model_gb.fit(X_train, y_train, save_checkpoint=False)
trained_model_rf = model_rf.fit(X_train, y_train, save_checkpoint=False)

### Evaluate the trained models

In [ ]:
# Gradient Boosting Regression model
y_true = y_test
y_pred = trained_model_gb.predict(X_test)
eval_regression_model(y_true, y_pred)

In [ ]:
# Random Forest Regression model
y_true = y_test
y_pred = trained_model_rf.predict(X_test)
eval_regression_model(y_true, y_pred)

## Pytorch Neural Network

In [ ]:
import torch
import torch.nn as nn

from utils.models import NeuralNetwork

In [ ]:
train_loader, test_loader = air_data_loader.get_data_loader_pytorch(batch_size_train=1024, 
                                                                    batch_size_test=len(air_data_loader.test_set))

In [ ]:
net = NeuralNetwork(num_input_feat=2)
loss_function = nn.L1Loss() # MAE Loss
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
num_epochs = 100

In [ ]:
for epoch in range(num_epochs):
    current_loss = 0.0
    print("Train epoch: {epoch}/{num_epochs}".format(epoch=epoch, num_epochs=num_epochs))
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()

        optimizer.step()

        current_loss += loss.item()
        if i % 10 == 0:
            current_loss = 0.0
    print("Test epoch ", epoch)
    for _, (inputs, targets) in enumerate(test_loader):
        y_true = targets.float().detach().numpy()
        y_pred = net(inputs.float()).detach().numpy()
        r2, mae, mse, rmse, mape, mdape = eval_regression_model(y_true=y_true, y_pred=y_pred, verbose=0)
    print("-------------------------------")

# torch.save(net.state_dict(), "trained_models/mlp.pth")